In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI(openai_api_key=api_key,temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 도움을 주는 AI 입니다. 주어지는 문서를 통해 질문에 대한 대답을 해주세요. 만약 문서에 정보가 없어서 대답을 하지 못한다면, 직접 답변을 만들지 말고 '모르겠습니다.' 라고 답변해주세요. 한국어로 대답해주세요. :\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("Victory Mansions 을 묘사해주세요.")

AIMessage(content='Victory Mansions은 오지니아의 주요 도시인 런던에 위치한 건물로, 그림자가 드리운 땅 위에 흰색으로 타워 형태로 솟아있는 건물입니다. 이 건물은 빅 브라더의 포스터가 걸린 복도를 통해 접근할 수 있으며, 전기가 낮에 차단되는 경제적 이유로 엘리베이터가 자주 작동하지 않는 것으로 알려져 있습니다. 건물은 7층으로 이루어져 있고, 건물 내부는 산란한 케이비지와 오래된 랙 매트의 냄새가 나는 복도로 시작됩니다.')